In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt

In [2]:
# Step 1: Load Data
def load_data(filepath):
    data = pd.read_csv(filepath)
    return data

In [3]:
def main():
    filepath = r"C:\\Users\\HP\\Documents\\GitHub\\intrusion-detection-fewshot-vs-traditional\\cleaned_dataset.csv"  
    data = load_data(filepath)

if __name__ == "__main__":
    main()